In [18]:
import json
from my_utils import NERDataset,collate_fn
from torch.utils.data import Dataset,DataLoader

In [2]:
dataset = NERDataset('nlp2024-data/dataset/dev.json')
dataloader = DataLoader(dataset,batch_size=8,collate_fn=collate_fn)
for batch in dataloader:
    print(batch)
    break

([[2584, 825, 2075, 1147, 1814, 1019, 122, 1515, 1147, 2448, 737, 2670, 926, 2448, 737, 2670, 1983, 970, 899, 564, 2075, 899, 1518, 737], [1147, 1814, 2670, 570, 2501, 2485, 737, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'], [2670, 2738, 1123, 2696, 2448, 737, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'], [2670, 2959, 2501, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'], [1983, 2738, 870, 2075, 864, 1983, 926, 899, 1518, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'], [2747, 1542, 2670, 2093, 210, 2448, 737, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'], [2670, 1983, 1549, 2604, 2696, 1455, 737, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'], [1983, 2670, '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#'

In [20]:
## 获取所有字符
data_dev = json.load(open('nlp2024-data/dataset/dev.json',encoding='utf-8'))
data_train = json.load(open('nlp2024-data/dataset/train.json',encoding='utf-8'))
def get_text_set(data):
    text_set = set()
    for key,val in data.items():
        for character in val['diagnosis']:
            text_set.add(character)
        for character in val['self_report']:
            text_set.add(character)
        dialogues = val['dialogue']
        for dia in dialogues:
            for character in dia['speaker']:
                text_set.add(character)
            for character in dia['sentence']:
                text_set.add(character)
            for word in dia['symptom_norm']:
                for character in word:
                    text_set.add(character)
    return text_set
text_set_train = get_text_set(data_train)
text_set_dev = get_text_set(data_dev)
text_set = text_set_dev | text_set_train
len(text_set)
text_list = list(text_set)
text_list.append('#')
text_dict = {text_list[i]:i for i in range(len(text_list))}
with open('char2id.json','w',encoding='utf-8') as f:
    json.dump(text_dict,f,ensure_ascii=False)

In [4]:
from transformers import BertModel,BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertModel.from_pretrained('bert-base-chinese')

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

In [12]:
text = '你好'
inputs = tokenizer(text,return_tensors='pt',padding=True,truncation=True)
inputs

{'input_ids': tensor([[ 101,  872, 1962,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [13]:
out = model(**inputs)

In [16]:
out.last_hidden_state.shape

torch.Size([1, 4, 768])

In [19]:
from my_utils import NERDatasetBert, collate_fn_bert
datasetBert = NERDatasetBert('nlp2024-data/dataset/dev.json',tokenizer)
dataloader = DataLoader(datasetBert,batch_size=2, collate_fn=collate_fn_bert)


In [21]:
for batch in dataloader:
    print(batch)
    break

(tensor([[ 101,  872, 1962, 8024, 1495, 1644, 3221, 6825, 1898, 1495, 1408, 8043,
         3300, 4588, 1408, 8043, 3300, 3766, 3837, 7965, 3873, 8024, 7965, 1853,
         8043,  102],
        [ 101, 1495, 1644, 3300, 1126, 1921,  749, 8043,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0]]), tensor([[ 101,  872, 1962, 8024, 1495, 1644, 3221, 6825, 1898, 1495, 1408, 8043,
         3300, 4588, 1408, 8043, 3300, 3766, 3837, 7965, 3873, 8024, 7965, 1853,
         8043,  102],
        [ 101, 1495, 1644, 3300, 1126, 1921,  749, 8043,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0]]), tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.]]))
